In [2]:
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import json
from bson import ObjectId
from pprint import pprint
import seaborn as sns
import pickle
import ast

In [3]:
with open('credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']

FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'

In [ ]:
client = MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))
db = client.rais

In [ ]:
# read sema collection
collection = db.sema
df = pd.DataFrame(list(collection.find()))

In [ ]:
# Deleting the no-responses and nones
df=pd.DataFrame(list(collection.find( { '$and': [ {"data.MOOD": { "$ne": "<no-response>" }},{"data.MOOD": { "$ne": None }}, {"data.PLACE": { "$ne": "<no-response>" }},{"data.PLACE": { "$ne": None }} ] } )))

In [ ]:
df=df[['user_id']].join(pd.DataFrame(df['data'].tolist(), index=df.index).add_prefix('data.'))

In [ ]:
df['date'] = pd.to_datetime(df['data.CREATED_TS']).dt.date
df['Time'] = pd.to_datetime(df['data.CREATED_TS']).dt.time
users=df['user_id']
days=df['date']
mood=df['data.MOOD']
df = pd.concat([users, days, mood],axis=1)
df = df.sort_values(by='date', ascending=True)
df['date'] = pd.to_datetime(df['date'].astype("str"), format='%Y-%m-%d')
df

In [ ]:
# Selecting experiment dates
df1 = df[(df['date'] >= "2021-05-24") & (df['date'] <= "2021-07-26")] #Round1
df2 = df[(df['date'] >= "2021-11-15") & (df['date'] <= "2022-01-17")] #Round2
df=pd.concat([df1, df2], ignore_index=True)
df

In [ ]:
sema = pd.get_dummies(df['data.MOOD'])
sema

In [ ]:
# combine one-hot encoding with actual df
sema_stress = pd.concat([df, sema], axis=1)
sema_stress.drop(['data.MOOD'], axis=1, inplace=True)

In [ ]:
sema_stress

In [ ]:
sema_stress.to_pickle('data/sema_stress_read_from_the_base_experiment_dates.pkl')